In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
from collections import Counter

In [2]:
# get environmental file
dirpath = "/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200"
envfile = os.path.join(dirpath, "cm1out_only_upto35.nc")
nc = xr.open_dataset(envfile)

In [3]:
# Converts time [ns] in time [seconds]
nc['time'] = nc['time'].values / np.timedelta64(1, 's')
nc['time'] = nc['time'].astype(np.int32)
time = nc['time'].values
print(time)
print(len(time))
print(len(np.unique(time)))
np.sort(time)

[5160 5220 5280 5340 5400 5460 5520 5580 5640 5700 5760 5820 5880 5940
 6000 6060 6120 6180 6240 6300 6360 6420 6480 6540 6600 6660 6720 6780
 6840 6900 6960 7020 7080 7140 7200 3660 3720 3780 3840 3900 3960 4020
 4080 4140 4200 4260 4320 4380 4440 4500 4560 4620 4680 4740 4800 4860
 4920 4980 5040 5100 5160 5220 5280 5340 5400 5460 5520 5580 5640 5700
 5760 5820 5880 5940 6000 6060 6120 6180 6240 6300 6360 6420 6480 6540
 6600 6660 6720 6780 6840 6900 6960 7020 7080 7140 7200 7260 7320 7380
 7440 7500 7560 7620 7680 7740 7800]
105
70


array([3660, 3720, 3780, 3840, 3900, 3960, 4020, 4080, 4140, 4200, 4260,
       4320, 4380, 4440, 4500, 4560, 4620, 4680, 4740, 4800, 4860, 4920,
       4980, 5040, 5100, 5160, 5160, 5220, 5220, 5280, 5280, 5340, 5340,
       5400, 5400, 5460, 5460, 5520, 5520, 5580, 5580, 5640, 5640, 5700,
       5700, 5760, 5760, 5820, 5820, 5880, 5880, 5940, 5940, 6000, 6000,
       6060, 6060, 6120, 6120, 6180, 6180, 6240, 6240, 6300, 6300, 6360,
       6360, 6420, 6420, 6480, 6480, 6540, 6540, 6600, 6600, 6660, 6660,
       6720, 6720, 6780, 6780, 6840, 6840, 6900, 6900, 6960, 6960, 7020,
       7020, 7080, 7080, 7140, 7140, 7200, 7200, 7260, 7320, 7380, 7440,
       7500, 7560, 7620, 7680, 7740, 7800], dtype=int32)

In [4]:
# Find duplicate timesteps
counts = Counter(time)
duplicates = [key for key, value in counts.items() if value > 1]
print("Duplicate values:", duplicates)
print(f'-> There are {len(duplicates)} timesteps')

Duplicate values: [5160, 5220, 5280, 5340, 5400, 5460, 5520, 5580, 5640, 5700, 5760, 5820, 5880, 5940, 6000, 6060, 6120, 6180, 6240, 6300, 6360, 6420, 6480, 6540, 6600, 6660, 6720, 6780, 6840, 6900, 6960, 7020, 7080, 7140, 7200]
-> There are 35 timesteps


In [5]:
# Find relative percent different for variables and put in df
data = [] 
vars = ['rh', 'th', 'prs', 'qi']
for i in duplicates:
    ds = nc.sel(time=i)
    ds1 = ds.isel(time=0)
    ds2 = ds.isel(time=1)
    record = []
    for var in vars:
        mean1 = ds1[var].mean()
        mean2 = ds2[var].mean()
        percent_diff = abs((mean2-mean1)/mean1)*100
        percent_diff = percent_diff.item()
        record.append(percent_diff)
    data.append(record)
df = pd.DataFrame(data, columns=vars)
df['time'] = duplicates
df

,rh,th,prs,qi,time
0,0.066126,0.000181,0.000180,2.148687,5160
1,0.066753,0.000152,0.000139,2.116559,5220
2,0.066454,0.000171,0.000131,2.063152,5280
3,0.065722,0.000190,0.000131,1.992787,5340
4,0.064705,0.000181,0.000082,1.910346,5400
5,0.063113,0.000190,0.000213,1.809863,5460
6,0.060807,0.000162,0.000098,1.686339,5520
7,0.057498,0.000171,0.000139,1.543838,5580
8,0.053591,0.000124,0.000139,1.375479,5640
9,0.049106,0.000152,0.000106,1.187827,5700
